In [ ]:
쥬피터노트북으로 실행, 최근정렬, 특정 키워

In [ ]:
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

# 1️⃣ 탐색할 경로 목록 (필요시 여기에 다른 경로 추가)
search_dirs = [
    Path.home() / "Documents",
    Path.home() / "Downloads",
    Path.home() / "Desktop"
]

# 2️⃣ 찾을 키워드 목록 (여기에 원하는 단어 자유롭게 추가)
keywords = ['해지', '관리', '시설']
extensions = ['.xlsx', '.xls', '.xlsm']
results = []

# 3️⃣ 탐색 실행
for search_dir in search_dirs:
    for root, dirs, files in os.walk(search_dir):
        for file in files:
            # 파일명에 키워드 중 하나라도 포함되어 있고 확장자가 엑셀인 경우
            if any(kw in file for kw in keywords) and Path(file).suffix in extensions:
                filepath = Path(root) / file
                mtime = datetime.fromtimestamp(os.path.getmtime(filepath))
                results.append({'파일명': file, '경로': str(filepath), '최종수정일시': mtime})

# 4️⃣ 결과 정리 및 정렬
df = pd.DataFrame(results)
df = df.sort_values(by='최종수정일시', ascending=False).reset_index(drop=True)

# 5️⃣ 결과 표시
import ace_tools as tools; tools.display_dataframe_to_user(name="엑셀 최근 파일 (해지/관리/시설 포함)", dataframe=df)

In [ ]:
import os
from pathlib import Path
import pandas as pd
from datetime import datetime

# ✅ 1. 검색할 드라이브 경로
search_dir = Path("D:/")

# ✅ 2. 키워드 및 확장자 설정
keywords = ['해지', '관리', '시설']
extensions = ['.xlsx', '.xls', '.xlsm']

# ✅ 3. 결과 저장용 리스트
results = []

# ✅ 4. 전체 탐색
for root, dirs, files in os.walk(search_dir):
    for file in files:
        try:
            if any(keyword in file for keyword in keywords) and Path(file).suffix.lower() in extensions:
                filepath = Path(root) / file
                mtime = os.path.getmtime(filepath)
                results.append({
                    '파일명': file,
                    '경로': str(filepath),
                    '최종수정일시': datetime.fromtimestamp(mtime)
                })
        except Exception as e:
            # 권한 오류 등은 무시
            continue

# ✅ 5. 결과 정리
if results:
    df = pd.DataFrame(results)
    df = df.sort_values(by='최종수정일시', ascending=False).reset_index(drop=True)
else:
    df = pd.DataFrame(columns=['파일명', '경로', '최종수정일시'])

# ✅ 6. 결과 표시 (필요시 저장도 가능)
df.head(20)  # 최근 20개만 미리보기로 출력

In [ ]:
# 가로로 펼침
import pandas as pd
from openpyxl import load_workbook

# 1. 파일 경로 설정
main_path = 'main.xlsx'
voc_path = 'voc.xlsx'
output_path = 'main_결과VOC.xlsx'

# 2. 엑셀 파일 읽기
df_main = pd.read_excel(main_path, sheet_name='Sheet1')
df_voc = pd.read_excel(voc_path, sheet_name='Sheet1')

# 3. VOC 파일에서 필요한 컬럼만 사용
voc_columns = ['계약번호', '접수일자', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_columns]

# 4. 접수일자 기준 정렬
df_voc = df_voc.sort_values(by=['계약번호', '접수일자'])

# 5. 계약번호 기준 VOC 다건 펼치기
merged_rows = []

for contract_no, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract_no}
    for idx, (_, item) in enumerate(group.iterrows(), 1):
        row[f'접수일자_{idx}'] = item['접수일자']
        row[f'VOC유형중_{idx}'] = item['VOC유형중']
        row[f'해지상세_{idx}'] = item['해지상세']
        row[f'등록내용_{idx}'] = item['등록내용']
        row[f'처리자_{idx}'] = item['처리자']
        row[f'처리내용_{idx}'] = item['처리내용']
    merged_rows.append(row)

df_voc_merged = pd.DataFrame(merged_rows)

# 6. 원본 메인 파일과 계약번호 기준 병합
df_joined = pd.merge(df_main, df_voc_merged, on='계약번호', how='left')

# 7. 기존 main.xlsx 불러와서 시트 추가 저장
with pd.ExcelWriter(output_path, engine='openpyxl', mode='w') as writer:
    # 기존 메인 시트 복사
    df_main.to_excel(writer, sheet_name='원본', index=False)
    # VOC 병합 결과 시트 추가
    df_joined.to_excel(writer, sheet_name='VOC_상세', index=False)

print(f"✅ 완료: '{output_path}'로 저장되었습니다.")

In [ ]:
import pandas as pd

# 파일 경로
main_path = 'main.xlsx'
voc_path = 'voc.xlsx'

# 엑셀 파일 읽기
df_main = pd.read_excel(main_path, sheet_name='Sheet1')
df_voc = pd.read_excel(voc_path, sheet_name='Sheet1')

# 컬럼명 정리 (voc 파일에서 필요한 컬럼만 추출)
voc_columns = ['계약번호', '접수일자', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_columns]

# 접수일자 기준 오름차순 정렬
df_voc = df_voc.sort_values(by=['계약번호', '접수일자'])

# 계약번호 기준으로 VOC 병합
merged_data = df_voc.groupby('계약번호').apply(
    lambda group: pd.Series({
        '접수일자': '\n'.join(group['접수일자'].astype(str).tolist()),
        'VOC유형중': '\n'.join(group['VOC유형중'].astype(str).tolist()),
        '해지상세': '\n'.join(group['해지상세'].astype(str).tolist()),
        '등록내용': '\n'.join(group['등록내용'].astype(str).tolist()),
        '처리자': '\n'.join(group['처리자'].astype(str).tolist()),
        '처리내용': '\n'.join(group['처리내용'].astype(str).tolist())
    })
).reset_index()

# 메인 데이터프레임에 병합
df_result = pd.merge(df_main, merged_data, how='left', on='계약번호')

# 결과 저장
df_result.to_excel('결과_병합파일.xlsx', index=False)

In [ ]:
---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
Cell In[1], line 11
      8 output_path = 'main_결과VOC.xlsx'
     10 # 2. 엑셀 파일 읽기
---> 11 df_main = pd.read_excel(main_path, sheet_name='db')
     12 df_voc = pd.read_excel(voc_path, sheet_name='VOC(해지및전체)병합202409-202504 - 복사본')
     14 # 3. VOC 파일에서 필요한 컬럼만 사용

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\io\excel\_base.py:495, in read_excel(io, sheet_name, header, names, index_col, usecols, dtype, engine, converters, true_values, false_values, skiprows, nrows, na_values, keep_default_na, na_filter, verbose, parse_dates, date_parser, date_format, thousands, decimal, comment, skipfooter, storage_options, dtype_backend, engine_kwargs)
    493 if not isinstance(io, ExcelFile):
    494     should_close = True
--> 495     io = ExcelFile(
    496         io,
    497         storage_options=storage_options,
    498         engine=engine,
    499         engine_kwargs=engine_kwargs,
    500     )
    501 elif engine and engine != io.engine:
    502     raise ValueError(
    503         "Engine should not be specified when passing "
    504         "an ExcelFile - ExcelFile already has the engine set"
    505     )

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\io\excel\_base.py:1550, in ExcelFile.__init__(self, path_or_buffer, engine, storage_options, engine_kwargs)
   1548     ext = "xls"
   1549 else:
-> 1550     ext = inspect_excel_format(
   1551         content_or_path=path_or_buffer, storage_options=storage_options
   1552     )
   1553     if ext is None:
   1554         raise ValueError(
   1555             "Excel file format cannot be determined, you must specify "
   1556             "an engine manually."
   1557         )

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\io\excel\_base.py:1402, in inspect_excel_format(content_or_path, storage_options)
   1399 if isinstance(content_or_path, bytes):
   1400     content_or_path = BytesIO(content_or_path)
-> 1402 with get_handle(
   1403     content_or_path, "rb", storage_options=storage_options, is_text=False
   1404 ) as handle:
   1405     stream = handle.handle
   1406     stream.seek(0)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\io\common.py:882, in get_handle(path_or_buf, mode, encoding, compression, memory_map, is_text, errors, storage_options)
    873         handle = open(
    874             handle,
    875             ioargs.mode,
   (...)
    878             newline="",
    879         )
    880     else:
    881         # Binary mode
--> 882         handle = open(handle, ioargs.mode)
    883     handles.append(handle)
    885 # Convert BytesIO or file objects passed with an encoding

FileNotFoundError: [Errno 2] No such file or directory: 'main.xlsx'

In [ ]:
import pandas as pd
import win32com.client as win32
from openpyxl import load_workbook
from pathlib import Path

# 1. 엑셀 앱 실행 중인 인스턴스 가져오기
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 시트 이름으로 워크북과 워크시트 가져오기
# 사용자가 지정한 시트명
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

# 시트 이름을 가진 워크북과 워크시트 찾기
main_ws = None
voc_ws = None

for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 시트명을 가진 워크시트를 찾을 수 없습니다. 시트 이름을 다시 확인해 주세요.")

# 3. 데이터 읽기
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 4. VOC 컬럼 필터링 및 정렬
voc_cols = ['계약번호', '접수일자', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols]
df_voc = df_voc.sort_values(by=['계약번호', '접수일자'])

# 5. 계약번호 기준 VOC 가로 펼치기
merged_rows = []
for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일자_{i}'] = r['접수일자']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 6. 병합
df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 7. 기존 워크북 경로 가져오기
output_path = Path(main_wb.FullName)

# 8. 병합 결과를 새로운 시트로 저장
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_result.to_excel(writer, sheet_name='VOC_상세', index=False)

print(f"✅ 병합 완료: '{output_path.name}' 파일에 'VOC_상세' 시트가 추가되었습니다.")

In [ ]:
import pandas as pd
import win32com.client as win32
from openpyxl import load_workbook
from pathlib import Path

# 1. Excel 실행 중인 인스턴스 연결
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 열려 있는 시트 이름 기준으로 가져오기
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

main_ws = None
voc_ws = None

for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 지정한 시트를 찾을 수 없습니다. 시트 이름 확인 필요.")

# 3. 시트 데이터를 DataFrame으로 변환
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 4. VOC 컬럼명 정리 및 사용 컬럼 필터링
df_voc.columns = df_voc.columns.str.strip()
df_main.columns = df_main.columns.str.strip()

voc_cols = ['계약번호', '접수일시', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols]
df_voc = df_voc.sort_values(by=['계약번호', '접수일시'])

# 5. 계약번호 기준 VOC를 가로로 펼치기
merged_rows = []

for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일시_{i}'] = r['접수일시']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 6. '계약번호'를 문자열로 통일 후 병합
df_main['계약번호'] = df_main['계약번호'].astype(str)
df_voc_flat['계약번호'] = df_voc_flat['계약번호'].astype(str)

df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 7. 원본 파일 경로 가져오기
output_path = Path(main_wb.FullName)

# 8. 결과를 'VOC_상세' 시트로 저장
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_result.to_excel(writer, sheet_name='VOC_상세', index=False)

print(f"✅ 병합 완료! '{output_path.name}' 파일에 'VOC_상세' 시트가 추가되었습니다.")

In [ ]:
import pandas as pd
import win32com.client as win32

# 1. Excel 실행 중인 인스턴스 연결
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 시트명 기준으로 워크시트 가져오기
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

main_ws, voc_ws = None, None

for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 지정한 시트를 찾지 못했습니다. 시트명을 확인해 주세요.")

# 3. UsedRange → DataFrame 변환
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 4. 컬럼명 정리 및 선택
df_main.columns = df_main.columns.str.strip()
df_voc.columns = df_voc.columns.str.strip()

voc_cols = ['계약번호', '접수일시', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols].sort_values(by=['계약번호', '접수일시'])

# 5. 계약번호별 VOC 이력 펼치기
merged_rows = []
for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일시_{i}'] = r['접수일시']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 6. 병합: 계약번호를 문자열로 통일
df_main['계약번호'] = df_main['계약번호'].astype(str)
df_voc_flat['계약번호'] = df_voc_flat['계약번호'].astype(str)
df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 7. 결과를 새 시트에 쓰기 (엑셀 직접 조작)
new_sheet_name = "VOC_상세"

# 기존 시트가 있다면 삭제
try:
    main_wb.Sheets(new_sheet_name).Delete()
except:
    pass  # 없으면 무시

# 새 시트 추가 및 데이터 입력
new_ws = main_wb.Sheets.Add()
new_ws.Name = new_sheet_name

# 데이터 쓰기
for i, col in enumerate(df_result.columns, start=1):
    new_ws.Cells(1, i).Value = col  # 헤더
for row_idx, row in enumerate(df_result.values, start=2):
    for col_idx, val in enumerate(row, start=1):
        new_ws.Cells(row_idx, col_idx).Value = val

print("✅ 병합 완료! 열린 엑셀 파일에 'VOC_상세' 시트가 직접 추가되었습니다.")

In [ ]:
AttributeError                            Traceback (most recent call last)
Cell In[15], line 75
     73 for i, col in enumerate(df_result.columns, start=1):
     74     new_ws.Cells(1, i).Value = col  # 헤더
---> 75 for row_idx, row in enumerate(df_result.values, start=2):
     76     for col_idx, val in enumerate(row, start=1):
     77         new_ws.Cells(row_idx, col_idx).Value = val

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\generic.py:6299, in NDFrame.__getattr__(self, name)
   6292 if (
   6293     name not in self._internal_names_set
   6294     and name not in self._metadata
   6295     and name not in self._accessors
   6296     and self._info_axis._can_hold_identifiers_and_holds_name(name)
   6297 ):
   6298     return self[name]
-> 6299 return object.__getattribute__(self, name)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\frame.py:12664, in DataFrame.values(self)
  12590 @property
  12591 def values(self) -> np.ndarray:
  12592     """
  12593     Return a Numpy representation of the DataFrame.
  12594
   (...)
  12662            ['monkey', nan, None]], dtype=object)
  12663     """
> 12664     return self._mgr.as_array()

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\internals\managers.py:1694, in BlockManager.as_array(self, dtype, copy, na_value)
   1692         arr.flags.writeable = False
   1693 else:
-> 1694     arr = self._interleave(dtype=dtype, na_value=na_value)
   1695     # The underlying data was copied within _interleave, so no need
   1696     # to further copy if copy=True or setting na_value
   1698 if na_value is lib.no_default:

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\internals\managers.py:1735, in BlockManager._interleave(self, dtype, na_value)
   1733 for blk in self.blocks:
   1734     rl = blk.mgr_locs
-> 1735     arr = blk.get_values(dtype)
   1736     result[rl.indexer] = arr
   1737     itemmask[rl.indexer] = 1

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:2251, in EABackedBlock.get_values(self, dtype)
   2249 values: ArrayLike = self.values
   2250 if dtype == _dtype_obj:
-> 2251     values = values.astype(object)
   2252 # TODO(EA2D): reshape not needed with 2D EAs
   2253 return np.asarray(values).reshape(self.shape)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\datetimes.py:739, in DatetimeArray.astype(self, dtype, copy)
    737 elif isinstance(dtype, PeriodDtype):
    738     return self.to_period(freq=dtype.freq)
--> 739 return dtl.DatetimeLikeArrayMixin.astype(self, dtype, copy)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\datetimelike.py:460, in DatetimeLikeArrayMixin.astype(self, dtype, copy)
    457     # *much* faster than self._box_values
    458     #  for e.g. test_get_loc_tuple_monotonic_above_size_cutoff
    459     i8data = self.asi8
--> 460     converted = ints_to_pydatetime(
    461         i8data,
    462         tz=self.tz,
    463         box="timestamp",
    464         reso=self._creso,
    465     )
    466     return converted
    468 elif self.dtype.kind == "m":

File vectorized.pyx:124, in pandas._libs.tslibs.vectorized.ints_to_pydatetime()

File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'

In [ ]:
import pandas as pd
import win32com.client as win32

# 1. 실행 중인 Excel 인스턴스 가져오기
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 사용할 시트 이름 정의
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

main_ws = voc_ws = None

# 3. 시트 탐색
for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 시트를 찾지 못했습니다. 이름을 다시 확인해 주세요.")

# 4. 시트 데이터를 DataFrame으로 변환
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 5. 컬럼명 정리
df_main.columns = df_main.columns.str.strip()
df_voc.columns = df_voc.columns.str.strip()

# 6. VOC에서 필요한 컬럼만 사용
voc_cols = ['계약번호', '접수일시', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols].sort_values(by=['계약번호', '접수일시'])

# 7. 계약번호별 VOC 다건을 가로로 펼치기
merged_rows = []
for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일시_{i}'] = r['접수일시']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 8. 계약번호 타입 통일 후 병합
df_main['계약번호'] = df_main['계약번호'].astype(str)
df_voc_flat['계약번호'] = df_voc_flat['계약번호'].astype(str)
df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 9. 'VOC_상세' 시트 새로 생성 및 작성
new_sheet_name = 'VOC_상세'

# 기존 시트 있으면 삭제
try:
    main_wb.Sheets(new_sheet_name).Delete()
except:
    pass

# 새 시트 추가 및 이름 설정
new_ws = main_wb.Sheets.Add()
new_ws.Name = new_sheet_name

# 10. 데이터 쓰기 (에러 방지 처리 포함)
for i, col in enumerate(df_result.columns, start=1):
    new_ws.Cells(1, i).Value = str(col)

for row_idx, row in enumerate(df_result.itertuples(index=False), start=2):
    for col_idx, val in enumerate(row, start=1):
        try:
            if pd.isna(val):
                new_ws.Cells(row_idx, col_idx).Value = ""
            else:
                new_ws.Cells(row_idx, col_idx).Value = str(val)
        except Exception as e:
            new_ws.Cells(row_idx, col_idx).Value = f"[ERROR: {e}]"

print("✅ 병합 완료: 열린 엑셀 파일에 'VOC_상세' 시트가 추가되었습니다.")

In [ ]:
import pandas as pd
import win32com.client as win32

# 1. Excel 실행 중인 인스턴스 연결
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 시트명 지정
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

main_ws = voc_ws = None

# 3. 시트 연결
for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 지정한 시트를 찾지 못했습니다. 시트명을 확인해 주세요.")

# 4. 시트 내용을 DataFrame으로 변환
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 5. 컬럼명 공백 제거
df_main.columns = df_main.columns.str.strip()
df_voc.columns = df_voc.columns.str.strip()

# 6. K열 계약번호 사용
# '계약번호'라는 이름이 여러 번 있을 수 있으므로 정확한 열 확인
# 아래 코드로 확인 후 필요한 경우 '계약번호.1' 등으로 변경
# print(df_main.columns.tolist())

# 필요 시 명확히 지정 (예: df_main['계약번호'] = df_main['계약번호.1'])

# 7. VOC 컬럼 필터링 및 정렬
voc_cols = ['계약번호', '접수일시', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols].sort_values(by=['계약번호', '접수일시'])

# 8. 계약번호 기준으로 VOC 이력 펼치기
merged_rows = []
for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일시_{i}'] = r['접수일시']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 9. 계약번호 문자열로 통일 후 병합
df_main['계약번호'] = df_main['계약번호'].astype(str)
df_voc_flat['계약번호'] = df_voc_flat['계약번호'].astype(str)

df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 10. 새 시트 'VOC_상세'로 결과 출력
new_sheet_name = 'VOC_상세'
try:
    main_wb.Sheets(new_sheet_name).Delete()
except:
    pass

new_ws = main_wb.Sheets.Add()
new_ws.Name = new_sheet_name

# 11. 문자열 변환 후 안전하게 엑셀에 쓰기
df_result_str = df_result.astype(str).fillna('')

for i, col in enumerate(df_result_str.columns, start=1):
    new_ws.Cells(1, i).Value = str(col)

for row_idx, row in enumerate(df_result_str.itertuples(index=False), start=2):
    for col_idx, val in enumerate(row, start=1):
        new_ws.Cells(row_idx, col_idx).Value = val

print("✅ 병합 완료! 열린 엑셀 파일에 'VOC_상세' 시트가 추가되었습니다.")

In [ ]:
--------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'
Exception ignored in: 'pandas._libs.tslibs.conversion._localize_tso'
Traceback (most recent call last):
  File "tzconversion.pyx", line 90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__
  File "timezones.pyx", line 305, in pandas._libs.tslibs.timezones.get_dst_info
AttributeError: 'NoneType' object has no attribute 'total_seconds'
Show more outputs








In [ ]:
import pandas as pd
import win32com.client as win32

# 1. 실행 중인 엑셀 인스턴스 연결
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 시트 이름 정의
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

main_ws = voc_ws = None

# 3. 시트 가져오기
for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 지정한 시트를 찾지 못했습니다. 시트명을 확인해 주세요.")

# 4. 시트 데이터를 DataFrame으로 변환
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 5. 컬럼명 공백 제거
df_main.columns = df_main.columns.str.strip()
df_voc.columns = df_voc.columns.str.strip()

# 6. 계약번호 컬럼 정확히 확인 (VOC는 D열 기준, DB는 K열 기준)
# 필요시 계약번호 위치 명확히 지정
# 예: df_main['계약번호'] = df_main['계약번호.1']

# 7. VOC 필터링 및 정렬
voc_cols = ['계약번호', '접수일시', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols].sort_values(by=['계약번호', '접수일시'])

# 8. 계약번호별 VOC 이력을 가로로 펼치기
merged_rows = []
for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일시_{i}'] = r['접수일시']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 9. 계약번호 타입 통일 후 병합
df_main['계약번호'] = df_main['계약번호'].astype(str)
df_voc_flat['계약번호'] = df_voc_flat['계약번호'].astype(str)

df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 10. 안전한 문자열 처리 (datetime, NaT, 오류 방지용)
def safe_str(val):
    try:
        if pd.isna(val):
            return ""
        return str(val)
    except:
        return "[ERROR]"

df_result_str = df_result.applymap(safe_str)

# 11. 'VOC_상세' 시트에 출력
new_sheet_name = 'VOC_상세'
try:
    main_wb.Sheets(new_sheet_name).Delete()
except:
    pass

new_ws = main_wb.Sheets.Add()
new_ws.Name = new_sheet_name

# 12. 엑셀에 값 쓰기
for i, col in enumerate(df_result_str.columns, start=1):
    new_ws.Cells(1, i).Value = str(col)

for row_idx, row in enumerate(df_result_str.itertuples(index=False), start=2):
    for col_idx, val in enumerate(row, start=1):
        new_ws.Cells(row_idx, col_idx).Value = val

print("✅ 병합 완료! 'VOC_상세' 시트가 열린 엑셀 파일에 성공적으로 추가되었습니다.")

In [ ]:
C:\Users\User\AppData\Local\Temp\ipykernel_20996\102059998.py:75: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_result_str = df_result.applymap(safe_str)
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[3], line 75
     72     except:
     73         return "[ERROR]"
---> 75 df_result_str = df_result.applymap(safe_str)
     77 # 11. 'VOC_상세' 시트에 출력
     78 new_sheet_name = 'VOC_상세'

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\frame.py:10522, in DataFrame.applymap(self, func, na_action, **kwargs)
  10473 """
  10474 Apply a function to a Dataframe elementwise.
  10475
   (...)
  10515 1  5  5
  10516 """
  10517 warnings.warn(
  10518     "DataFrame.applymap has been deprecated. Use DataFrame.map instead.",
  10519     FutureWarning,
  10520     stacklevel=find_stack_level(),
  10521 )
> 10522 return self.map(func, na_action=na_action, **kwargs)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\frame.py:10468, in DataFrame.map(self, func, na_action, **kwargs)
  10465 def infer(x):
  10466     return x._map_values(func, na_action=na_action)
> 10468 return self.apply(infer).__finalize__(self, "map")

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\frame.py:10374, in DataFrame.apply(self, func, axis, raw, result_type, args, by_row, engine, engine_kwargs, **kwargs)
  10360 from pandas.core.apply import frame_apply
  10362 op = frame_apply(
  10363     self,
  10364     func=func,
   (...)
  10372     kwargs=kwargs,
  10373 )
> 10374 return op.apply().__finalize__(self, method="apply")

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\apply.py:916, in FrameApply.apply(self)
    913 elif self.raw:
    914     return self.apply_raw(engine=self.engine, engine_kwargs=self.engine_kwargs)
--> 916 return self.apply_standard()

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\apply.py:1063, in FrameApply.apply_standard(self)
   1061 def apply_standard(self):
   1062     if self.engine == "python":
-> 1063         results, res_index = self.apply_series_generator()
   1064     else:
   1065         results, res_index = self.apply_series_numba()

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\apply.py:1081, in FrameApply.apply_series_generator(self)
   1078 with option_context("mode.chained_assignment", None):
   1079     for i, v in enumerate(series_gen):
   1080         # ignore SettingWithCopy here in case the user mutates
-> 1081         results[i] = self.func(v, *self.args, **self.kwargs)
   1082         if isinstance(results[i], ABCSeries):
   1083             # If we have a view on v, we need to make a copy because
   1084             #  series_generator will swap out the underlying data
   1085             results[i] = results[i].copy(deep=False)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\frame.py:10466, in DataFrame.map.<locals>.infer(x)
  10465 def infer(x):
> 10466     return x._map_values(func, na_action=na_action)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\base.py:919, in IndexOpsMixin._map_values(self, mapper, na_action, convert)
    916 arr = self._values
    918 if isinstance(arr, ExtensionArray):
--> 919     return arr.map(mapper, na_action=na_action)
    921 return algorithms.map_array(arr, mapper, na_action=na_action, convert=convert)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\_mixins.py:81, in ravel_compat.<locals>.method(self, *args, **kwargs)
     78 @wraps(meth)
     79 def method(self, *args, **kwargs):
     80     if self.ndim == 1:
---> 81         return meth(self, *args, **kwargs)
     83     flags = self._ndarray.flags
     84     flat = self.ravel("K")

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\datetimelike.py:740, in DatetimeLikeArrayMixin.map(self, mapper, na_action)
    736 @ravel_compat
    737 def map(self, mapper, na_action=None):
    738     from pandas import Index
--> 740     result = map_array(self, mapper, na_action=na_action)
    741     result = Index(result)
    743     if isinstance(result, ABCMultiIndex):

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\algorithms.py:1741, in map_array(arr, mapper, na_action, convert)
   1738     return arr.copy()
   1740 # we must convert to python types
-> 1741 values = arr.astype(object, copy=False)
   1742 if na_action is None:
   1743     return lib.map_infer(values, mapper, convert=convert)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\datetimes.py:739, in DatetimeArray.astype(self, dtype, copy)
    737 elif isinstance(dtype, PeriodDtype):
    738     return self.to_period(freq=dtype.freq)
--> 739 return dtl.DatetimeLikeArrayMixin.astype(self, dtype, copy)

File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\datetimelike.py:460, in DatetimeLikeArrayMixin.astype(self, dtype, copy)
    457     # *much* faster than self._box_values
    458     #  for e.g. test_get_loc_tuple_monotonic_above_size_cutoff
    459     i8data = self.asi8
--> 460     converted = ints_to_pydatetime(
    461         i8data,
    462         tz=self.tz,
    463         box="timestamp",
    464         reso=self._creso,
    465     )
    466     return converted
    468 elif self.dtype.kind == "m":

File vectorized.pyx:124, in pandas._libs.tslibs.vectorized.ints_to_pydatetime()

File tzconversion.pyx:90, in pandas._libs.tslibs.tzconversion.Localizer.__cinit__()

File timezones.pyx:305, in pandas._libs.tslibs.timezones.get_dst_info()

AttributeError: 'NoneType' object has no attribute 'total_seconds'






In [ ]:
import pandas as pd
import win32com.client as win32

# 1. Excel 실행 중인 인스턴스 연결
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 사용할 시트 이름 정의
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

main_ws = voc_ws = None

# 3. 열려 있는 워크북에서 시트 찾기
for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 시트를 찾지 못했습니다. 'db'와 VOC 시트명을 확인해 주세요.")

# 4. 시트 데이터 읽기
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 5. 컬럼명 공백 제거
df_main.columns = df_main.columns.str.strip()
df_voc.columns = df_voc.columns.str.strip()

# 6. VOC에서 필요한 컬럼만 추출
voc_cols = ['계약번호', '접수일시', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols].sort_values(by=['계약번호', '접수일시'])

# 7. 계약번호별 VOC 다건을 가로로 펼치기
merged_rows = []
for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일시_{i}'] = r['접수일시']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 8. 계약번호를 문자열로 통일
df_main['계약번호'] = df_main['계약번호'].astype(str)
df_voc_flat['계약번호'] = df_voc_flat['계약번호'].astype(str)

# 9. 병합 수행
df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 10. 'VOC_상세' 시트 만들기 (기존 있으면 삭제)
new_sheet_name = 'VOC_상세'
try:
    main_wb.Sheets(new_sheet_name).Delete()
except:
    pass

new_ws = main_wb.Sheets.Add()
new_ws.Name = new_sheet_name

# 11. 컬럼명 작성
for i, col in enumerate(df_result.columns, start=1):
    new_ws.Cells(1, i).Value = str(col)

# 12. 데이터 삽입 (datetime, NaT, tz=None 등 모든 오류 예방)
for row_idx, row in enumerate(df_result.itertuples(index=False), start=2):
    for col_idx, val in enumerate(row, start=1):
        try:
            if pd.isna(val):
                new_ws.Cells(row_idx, col_idx).Value = ""
            elif hasattr(val, "strftime"):  # datetime 객체 처리
                new_ws.Cells(row_idx, col_idx).Value = val.strftime("%Y-%m-%d %H:%M:%S")
            else:
                new_ws.Cells(row_idx, col_idx).Value = str(val)
        except:
            new_ws.Cells(row_idx, col_idx).Value = "[ERR]"

print("✅ 병합 완료! 'VOC_상세' 시트가 엑셀에 정상 추가되었습니다.")

In [ ]:
import pandas as pd
import win32com.client as win32

# 1. Excel 인스턴스 연결
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 시트명 정의
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

main_ws = voc_ws = None

# 3. 열린 엑셀 워크북에서 시트 찾기
for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 시트를 찾을 수 없습니다. 시트 이름을 확인해 주세요.")

# 4. 데이터프레임 변환
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 5. 컬럼명 공백 제거
df_main.columns = df_main.columns.str.strip()
df_voc.columns = df_voc.columns.str.strip()

# ✅ 계약번호 정규화 함수 (모든 숫자를 문자열로 처리)
def normalize_contract(col):
    return col.astype(str).str.replace(r'\.0$', '', regex=True).str.strip()

df_main['계약번호'] = normalize_contract(df_main['계약번호'])
df_voc['계약번호'] = normalize_contract(df_voc['계약번호'])

# 6. VOC 컬럼 필터링 및 정렬
voc_cols = ['계약번호', '접수일시', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols].sort_values(by=['계약번호', '접수일시'])

# 7. 계약번호별 VOC 이력 펼치기
merged_rows = []
for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일시_{i}'] = r['접수일시']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 8. 병합
df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 9. 'VOC_상세' 시트 생성 (기존 있으면 삭제)
new_sheet_name = 'VOC_상세'
try:
    main_wb.Sheets(new_sheet_name).Delete()
except:
    pass

new_ws = main_wb.Sheets.Add()
new_ws.Name = new_sheet_name

# 10. 컬럼명 작성
for i, col in enumerate(df_result.columns, start=1):
    new_ws.Cells(1, i).Value = str(col)

# 11. 셀 데이터 안전하게 쓰기 (datetime 포함 처리)
for row_idx, row in enumerate(df_result.itertuples(index=False), start=2):
    for col_idx, val in enumerate(row, start=1):
        try:
            if pd.isna(val):
                new_ws.Cells(row_idx, col_idx).Value = ""
            elif hasattr(val, "strftime"):
                new_ws.Cells(row_idx, col_idx).Value = val.strftime("%Y-%m-%d %H:%M:%S")
            else:
                new_ws.Cells(row_idx, col_idx).Value = str(val)
        except:
            new_ws.Cells(row_idx, col_idx).Value = "[ERR]"

print("✅ 병합 완료! '

In [ ]:
import pandas as pd
import win32com.client as win32

# 1. 엑셀 인스턴스 연결
excel = win32.GetActiveObject("Excel.Application")
workbooks = excel.Workbooks

# 2. 시트명 정의
main_sheet_name = 'db'
voc_sheet_name = 'VOC(해지및전체)병합202409-202504 - 복사본'

main_ws = voc_ws = None

# 3. 시트 찾기
for wb in workbooks:
    for sheet in wb.Sheets:
        if sheet.Name == main_sheet_name:
            main_ws = sheet
            main_wb = wb
        elif sheet.Name == voc_sheet_name:
            voc_ws = sheet

if main_ws is None or voc_ws is None:
    raise ValueError("❌ 시트를 찾지 못했습니다. 시트 이름 확인 필요")

# 4. 데이터프레임 변환
main_data = main_ws.UsedRange.Value
voc_data = voc_ws.UsedRange.Value

df_main = pd.DataFrame(main_data[1:], columns=main_data[0])
df_voc = pd.DataFrame(voc_data[1:], columns=voc_data[0])

# 5. 컬럼 정리
df_main.columns = df_main.columns.str.strip()
df_voc.columns = df_voc.columns.str.strip()

# ✅ 6. 계약번호 정규화 함수
def normalize_contract(col):
    return col.astype(str).str.replace(r'\.0$', '', regex=True).str.strip()

df_main['계약번호'] = normalize_contract(df_main['계약번호'])
df_voc['계약번호'] = normalize_contract(df_voc['계약번호'])

# 7. VOC 필터링
voc_cols = ['계약번호', '접수일시', 'VOC유형중', '해지상세', '등록내용', '처리자', '처리내용']
df_voc = df_voc[voc_cols].sort_values(by=['계약번호', '접수일시'])

# 8. VOC 이력 가로 확장
merged_rows = []
for contract, group in df_voc.groupby('계약번호'):
    row = {'계약번호': contract}
    for i, (_, r) in enumerate(group.iterrows(), 1):
        row[f'접수일시_{i}'] = r['접수일시']
        row[f'VOC유형중_{i}'] = r['VOC유형중']
        row[f'해지상세_{i}'] = r['해지상세']
        row[f'등록내용_{i}'] = r['등록내용']
        row[f'처리자_{i}'] = r['처리자']
        row[f'처리내용_{i}'] = r['처리내용']
    merged_rows.append(row)

df_voc_flat = pd.DataFrame(merged_rows)

# 9. 병합
df_result = pd.merge(df_main, df_voc_flat, how='left', on='계약번호')

# 10. 결과 시트 작성
new_sheet_name = 'VOC_상세'
try:
    main_wb.Sheets(new_sheet_name).Delete()
except:
    pass

new_ws = main_wb.Sheets.Add()
new_ws.Name = new_sheet_name

# 11. 헤더 작성
for i, col in enumerate(df_result.columns, start=1):
    new_ws.Cells(1, i).Value = str(col)

# 12. 데이터 안전하게 입력
for row_idx, row in enumerate(df_result.itertuples(index=False), start=2):
    row_vals = list(row)
    for col_idx, val in enumerate(row_vals, start=1):
        try:
            if pd.isna(val):
                new_ws.Cells(row_idx, col_idx).Value = ""
            elif hasattr(val, "strftime"):
                new_ws.Cells(row_idx, col_idx).Value = val.strftime("%Y-%m-%d %H:%M:%S")
            else:
                new_ws.Cells(row_idx, col_idx).Value = str(val)
        except:
            new_ws.Cells(row_idx, col_idx).Value = "[ERR]"

print("✅ 병합 완료! 'VOC_상세' 시트에 정상적으로 데이터가 입력되었습니다.")